In [23]:
!/usr/bin/env python -m pip install selenium pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 2.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 4.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import time
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [3]:
firefox_options = webdriver.FirefoxOptions()
driver = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=firefox_options)

In [4]:
routes = []

In [5]:
driver.get(f"https://2gis.ru/moscow/search/электричка/")
for page in range(1, 15):
    routes.extend(el.find_element(By.CLASS_NAME, "_1rehek").get_attribute('href') for el in driver.find_elements(By.CLASS_NAME, "_1kf6gff") if el.find_element(By.CLASS_NAME, "_96iy63").text == "Электричка")
    driver.execute_script("document.querySelector('div._n5hmn94:last-child').click();")
    time.sleep(1.5)

NoSuchElementException: Message: Unable to locate element: ._96iy63
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


In [6]:
len(routes)
routes[0]

'https://2gis.ru/moscow/route/4504205217761251?stat=eyJwYXJlbnRUYWJJZCI6IjI5NzgyODdlLTRmNjEtNDMzNS1hY2Q1LTk0ZTUwZjgwNDc5MiIsImZvcmtFdmVudE9yZGluYWwiOjIsInVpRWxlbWVudCI6eyJuYW1lIjoicGxhY2VDYXJkTWluaSIsIm93bmVyTmFtZSI6InNlYXJjaFJlc3VsdHNMaXN0IiwicG9zaXRpb24iOjF9LCJwbGFjZUl0ZW0iOnsiZW50aXR5Ijp7ImlkIjoiNDUwNDIwNTIxNzc2MTI1MSIsInR5cGUiOiJyb3V0ZSIsInN1YnR5cGUiOiJzdWJ1cmJhbl90cmFpbiIsInNlZ21lbnRJbmZvIjp7ImJhc2VMb2NhbGUiOiJydV9SVSIsInNlZ21lbnRJZCI6IjMyIn19LCJyZXN1bHRDbGFzcyI6ODcsInBvc2l0aW9uIjoxfSwic2VhcmNoUmVzdWx0c0xpc3QiOnsic2l6ZSI6MTYyLCJzZWFyY2hUeXBlIjo3LCJmaXJzdFJlc3VsdENsYXNzIjo3LCJoYXNHZW9SZXN0cmljdGlvbiI6ZmFsc2UsInBhcnRpYWxSZXN1bHQiOmZhbHNlLCJvdXR2aWV3cG9ydCI6W3sibGF0Ijo1NS44NzgwODk1ODc4MTQzNDQsImxvbiI6MzcuMzAxNTY2NTAxMDc2NDJ9LHsibGF0Ijo1NS42Mjg4NDI0MTIxODU2NiwibG9uIjozNy4zMDE1NjY1MDEwNzY0Mn0seyJsYXQiOjU1LjYyODg0MjQxMjE4NTY2LCJsb24iOjM3LjkzODc3MzQ5ODkyMzU4fSx7ImxhdCI6NTUuODc4MDg5NTg3ODE0MzQ0LCJsb24iOjM3LjkzODc3MzQ5ODkyMzU4fV0sInNlYXJjaFNlZ21lbnRJZCI6IjMyIiwicnR3VmVyc2lvbiI6ImU1ZTQyZjBm

In [7]:
ROUTE_ID_REGEX = re.compile(r"route/(\d+)")
PLATFORM_ID_REGEX = re.compile(r"platform\/(\d+)")
platforms = []
for route_url in routes:
    route_id = ROUTE_ID_REGEX.search(route_url)[1]
    driver.get(route_url)
    platforms.extend(
        dict(route_id=route_id, platform_id=PLATFORM_ID_REGEX.search(a.get_attribute("href"))[1], platform_name=a.find_element(By.TAG_NAME, 'span').text, href=a.get_attribute("href"))
        for a in driver.find_element(By.CLASS_NAME, "_9xuavt").find_elements(By.CLASS_NAME, "_2lcm958")
    )
    # break

In [17]:
COORDS_REGEX = re.compile(r"points\/.?(?P<lon>\d+\.\d+),(?P<lat>\d+\.\d+);")
from urllib.parse import unquote
for p in platforms:
    driver.get(p.get("href"))
    href = driver.find_element(By.XPATH, "//div[@class='_vaj62s'][last()]/a[@class='_1qhm93s']").get_attribute("href")
    p.update(COORDS_REGEX.search(unquote(href)).groupdict())

In [20]:
import json
with open("./platforms.json", 'w') as f:
    json.dump(platforms, f)

In [24]:
import pandas as pd
df = pd.DataFrame.from_records(platforms)
df.head()

,route_id,platform_id,platform_name,href,lon,lat
0,4504205217761251,4504282527180955,Катуар,https://2gis.ru/moscow/platform/4504282527180955,37.504445,56.091081
1,4504205217761251,4504282527180951,Некрасовская,https://2gis.ru/moscow/platform/4504282527180951,37.495965,56.074599
2,4504205217761251,4504282527180909,Луговая,https://2gis.ru/moscow/platform/4504282527180909,37.492626,56.048837
3,4504205217761251,4504282834361776,Депо,https://2gis.ru/moscow/platform/4504282834361776,37.486449,56.028549
4,4504205217761251,4504282527180983,Лобня,https://2gis.ru/moscow/platform/4504282527180983,37.484698,56.013821


In [25]:
df.drop(columns=['href'], inplace=True)
df.lon = pd.to_numeric(df.lon)
df.lat = pd.to_numeric(df.lat)

In [32]:
df = df.groupby('platform_id').agg({'route_id': list, 'platform_name': 'first', 'lon': 'first', 'lat': 'first'}).reset_index()

In [33]:
import redis
R = redis.Redis(host='localhost')

In [35]:
for index, row in df.iterrows():
    data = row.to_dict()
    lon = data.pop("lon")
    lat = data.pop("lat")
    R.geoadd("organization:railroad", (lon, lat, json.dumps(data)))